In [1]:
import json
import nltk 
import string
import copy
import random
from os import listdir
import pickle



import numpy as np

import string
import re

from nltk.stem.porter import *
from nltk.corpus import stopwords
from TurkishStemmer import TurkishStemmer


from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier,LinearRegression
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import accuracy_score


from pos_tagger import tag

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
all_dicts = pickle.load( open( "./data/clean_data.p", "rb" ) )

article_text_dict_positive=all_dicts["article_text_dict_positive"]
iter1_BBC_text_dict_neg=all_dicts["iter1_BBC_text_dict_neg"]
iter2_BBC_text_dict_neg=all_dicts["iter2_BBC_text_dict_neg"]
iter1_CNN_neg_text=all_dicts["iter1_CNN_neg_text"]

In [3]:
print("",len(iter1_BBC_text_dict_neg))
print("",len(iter2_BBC_text_dict_neg))
print("",len(iter1_CNN_neg_text))
print("",len(article_text_dict_positive))

 1038
 1043
 948
 1037


In [4]:
lowercase=' abcdefghijklmnoprstuvyzğöıüşç'
stemmer = TurkishStemmer()

# There are different tokenization functions

# first one is tokenizes with nltk, lowers text 
# and applies stemming to each word
def tokenize_stem(text):
    """Removes punctuation & excess whitespace, sets to lowercase,
    and stems text. Returns a list of them"""
    text=nltk.word_tokenize(text.lower().strip())
    tokens = [stemmer.stem(t) for t in text]
    return tokens


# lowers text
# generates part of speech tags  
# returns "word+tag" 
# tokenization made with tagging library
def pos_tokenize(text):
    text=text.lower().strip()
    tokens=[]
    tags=tag(text)
    for a_tag in tags:
        tokens.append(a_tag[0]+"+"+a_tag[1])
    return tokens

# lowers text
# splits text to it's characters
def char_tokenize(text):
    text = text.lower().strip()
    tokens = [t for t in text]
    return tokens



In [5]:
# list of feature functions to be tested 
features={"Pos_tags":pos_tokenize,"characters":char_tokenize,"tokenize":nltk.word_tokenize,"tokenize_stem":tokenize_stem}


In [6]:
def run_experiment_w_features(X_train,X_test,y_train,y_test,feature,n_range,theclassifiers,vectorizer=None):
    if vectorizer==None:
        vectorizer = TfidfVectorizer(
            tokenizer= features[feature],
#             cleaning itself
#             preprocessor=word_process_clean,
            ngram_range=n_range,
            use_idf=True,
            min_df=0.003,
            norm=None, )
        
    
    X_train = vectorizer.fit_transform(X_train).toarray()
    X_test = vectorizer.transform(X_test).toarray()

    if type(theclassifiers)!=list:
        theclassifiers=[theclassifiers]
    for theclassifier in theclassifiers:
#       initialise classifier
        if theclassifier==LinearSVC:
            clf= theclassifier(max_iter=-1)
        else:
            clf= theclassifier()
#     create model with tranining data
        model = clf.fit(X_train, y_train)
#     predict test set
        y_preds = model.predict(X_test)
#     create the report
        report = classification_report( y_test, y_preds )
    
#     find name of the classifier for printing
        match=re.search(r"\.([A-z]*)'>",str(theclassifier))
        match=match.group(1)

        result_text="\033[1m Performance report of \033[0m \033[92m" + feature +"\033[0m "
        count=0
        for i in range(n_range[0],n_range[1]+1):
            if count!=0:
                result_text+=" and "
            result_text=result_text+"\033[91m"+str(i)+"-gram\033[0m"
            count+=1
        result_text+= " with \033[94m"+match+"\033[0m"
    #     print("Performance report of {} {}-gram".format(feature,n_range[1]))
        print (result_text)
#         prnt_scores(report)
        print(report)
        print(accuracy_score( y_test, y_preds))


In [7]:
pos_examp=[]
neg_examp_train=[]
neg_examp_test=[]

print("pos examples",len(article_text_dict_positive))
print("BBC",len(iter2_BBC_text_dict_neg))
print("CNN",len(iter1_CNN_neg_text))

# some articles includes short text 
for artc in article_text_dict_positive.values():
    pos_examp.append((artc,1))

for artc in iter2_BBC_text_dict_neg.values():
    neg_examp_train.append((artc,0))

for artc in iter1_CNN_neg_text.values():
    neg_examp_test.append((artc,0))

print("pos examples",len(pos_examp))
print("BBC",len(pos_examp))
print("CNN",len(neg_examp_test))

# combine positive and negative samples then shuffle
# XY=pos_examp+neg_examp_train
random.seed(a=2)
random.shuffle(pos_examp)
random.shuffle(neg_examp_train)
random.shuffle(neg_examp_test)

percentage=0.8
cut_point=int(len(pos_examp)*percentage)
train_pos=pos_examp[:cut_point]
test_pos=pos_examp[cut_point:]

cut_point=int(len(neg_examp_train)*percentage)
train_neg=neg_examp_train[:cut_point]
cut_point=int(len(neg_examp_test)*percentage)
test_neg=neg_examp_test[cut_point:]

print("train_pos",len(train_pos),"test_pos",len(test_pos))
print("train_neg",len(train_neg),"test_neg",len(test_neg))

XY_train=train_pos+train_neg
random.shuffle(XY_train)
X_train=[k[0] for k in XY_train]
Y_train=[k[1] for k in XY_train]

XY_test=test_pos+test_neg
random.shuffle(XY_test)
X_test=[k[0] for k in XY_test]
Y_test=[k[1] for k in XY_test]


print("X_train",len(X_train),"Y_train",len(Y_train),"X_test",len(X_test),"Y_test",len(Y_test))



pos examples 1037
BBC 1043
CNN 948
pos examples 1037
BBC 1037
CNN 948
train_pos 829 test_pos 208
train_neg 834 test_neg 190
X_train 1663 Y_train 1663 X_test 398 Y_test 398


In [8]:
classifiers_list=[LinearSVC,MultinomialNB,RandomForestClassifier,
                  AdaBoostClassifier]
tryfeatures={"tokenize":nltk.word_tokenize,"tokenize_stem":tokenize_stem,"Pos_tags":pos_tokenize,"characters":char_tokenize}



In [9]:
for clasifier in classifiers_list:
    for feature in tryfeatures:
        for i in range(1,4):
            for k in range(i,4):
                run_experiment_w_features(X_train[:],X_test[:],Y_train[:],Y_test[:],feature,(i,k),clasifier)


/anaconda3/envs/HPCenv/lib/python3.7/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/anaconda3/envs/HPCenv/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


 Performance report of  tokenize 1-gram with LinearSVC
              precision    recall  f1-score   support

           0       0.48      1.00      0.65       190
           1       0.00      0.00      0.00       208

   micro avg       0.48      0.48      0.48       398
   macro avg       0.24      0.50      0.32       398
weighted avg       0.23      0.48      0.31       398

0.47738693467336685


/anaconda3/envs/HPCenv/lib/python3.7/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/anaconda3/envs/HPCenv/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


 Performance report of  tokenize 1-gram and 2-gram with LinearSVC
              precision    recall  f1-score   support

           0       0.48      1.00      0.65       190
           1       0.00      0.00      0.00       208

   micro avg       0.48      0.48      0.48       398
   macro avg       0.24      0.50      0.32       398
weighted avg       0.23      0.48      0.31       398

0.47738693467336685


/anaconda3/envs/HPCenv/lib/python3.7/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/anaconda3/envs/HPCenv/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


...

/anaconda3/envs/HPCenv/lib/python3.7/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/anaconda3/envs/HPCenv/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


 Performance report of  characters 3-gram with LinearSVC
              precision    recall  f1-score   support

           0       0.48      1.00      0.65       190
           1       0.00      0.00      0.00       208

   micro avg       0.48      0.48      0.48       398
   macro avg       0.24      0.50      0.32       398
weighted avg       0.23      0.48      0.31       398

0.47738693467336685
 Performance report of  tokenize 1-gram with MultinomialNB
              precision    recall  f1-score   support

           0       0.91      0.88      0.90       190
           1       0.90      0.92      0.91       208

   micro avg       0.90      0.90      0.90       398
   macro avg       0.91      0.90      0.90       398
weighted avg       0.90      0.90      0.90       398

0.9045226130653267
 Performance report of  tokenize 1-gram and 2-gram with MultinomialNB
              precision    recall  f1-score   support

           0       0.93      0.85      0.89       190
           1

 Performance report of  characters 1-gram with MultinomialNB
              precision    recall  f1-score   support

           0       0.73      0.77      0.75       190
           1       0.78      0.75      0.76       208

   micro avg       0.76      0.76      0.76       398
   macro avg       0.76      0.76      0.76       398
weighted avg       0.76      0.76      0.76       398

0.7587939698492462
 Performance report of  characters 1-gram and 2-gram with MultinomialNB
              precision    recall  f1-score   support

           0       0.84      0.89      0.87       190
           1       0.90      0.84      0.87       208

   micro avg       0.87      0.87      0.87       398
   macro avg       0.87      0.87      0.87       398
weighted avg       0.87      0.87      0.87       398

0.8668341708542714
 Performance report of  characters 1-gram and 2-gram and 3-gram with MultinomialNB
              precision    recall  f1-score   support

           0       0.91      0.90    

 Performance report of  Pos_tags 1-gram with RandomForestClassifier
              precision    recall  f1-score   support

           0       0.85      0.83      0.84       190
           1       0.85      0.87      0.86       208

   micro avg       0.85      0.85      0.85       398
   macro avg       0.85      0.85      0.85       398
weighted avg       0.85      0.85      0.85       398

0.8492462311557789
 Performance report of  Pos_tags 1-gram and 2-gram with RandomForestClassifier
              precision    recall  f1-score   support

           0       0.83      0.81      0.82       190
           1       0.83      0.85      0.84       208

   micro avg       0.83      0.83      0.83       398
   macro avg       0.83      0.83      0.83       398
weighted avg       0.83      0.83      0.83       398

0.8266331658291457
 Performance report of  Pos_tags 1-gram and 2-gram and 3-gram with RandomForestClassifier
              precision    recall  f1-score   support

           0    

 Performance report of  tokenize_stem 1-gram with AdaBoostClassifier
              precision    recall  f1-score   support

           0       0.83      0.65      0.73       190
           1       0.73      0.88      0.80       208

   micro avg       0.77      0.77      0.77       398
   macro avg       0.78      0.77      0.77       398
weighted avg       0.78      0.77      0.77       398

0.7713567839195979
 Performance report of  tokenize_stem 1-gram and 2-gram with AdaBoostClassifier
              precision    recall  f1-score   support

           0       0.83      0.65      0.73       190
           1       0.73      0.88      0.80       208

   micro avg       0.77      0.77      0.77       398
   macro avg       0.78      0.77      0.77       398
weighted avg       0.78      0.77      0.77       398

0.7713567839195979
 Performance report of  tokenize_stem 1-gram and 2-gram and 3-gram with AdaBoostClassifier
              precision    recall  f1-score   support

           0 

In [10]:
slow_classifiers_list=[DecisionTreeClassifier,GaussianNB,MLPClassifier]
for clasifier in slow_classifiers_list:
    for feature in tryfeatures:
        for i in range(1,4):
            run_experiment_w_features(X_train[:],X_test[:],Y_train[:],Y_test[:],feature,(i,i),clasifier)

 Performance report of  tokenize 1-gram with DecisionTreeClassifier
              precision    recall  f1-score   support

           0       0.78      0.58      0.67       190
           1       0.69      0.85      0.76       208

   micro avg       0.72      0.72      0.72       398
   macro avg       0.74      0.72      0.72       398
weighted avg       0.73      0.72      0.72       398

0.7236180904522613
 Performance report of  tokenize 2-gram with DecisionTreeClassifier
              precision    recall  f1-score   support

           0       0.66      0.49      0.56       190
           1       0.62      0.77      0.69       208

   micro avg       0.64      0.64      0.64       398
   macro avg       0.64      0.63      0.63       398
weighted avg       0.64      0.64      0.63       398

0.6381909547738693
 Performance report of  tokenize 3-gram with DecisionTreeClassifier
              precision    recall  f1-score   support

           0       0.72      0.25      0.37      

 Performance report of  Pos_tags 2-gram with GaussianNB
              precision    recall  f1-score   support

           0       0.79      0.46      0.58       190
           1       0.64      0.89      0.75       208

   micro avg       0.68      0.68      0.68       398
   macro avg       0.72      0.67      0.66       398
weighted avg       0.71      0.68      0.67       398

0.6834170854271356
 Performance report of  Pos_tags 3-gram with GaussianNB
              precision    recall  f1-score   support

           0       0.63      0.14      0.23       190
           1       0.54      0.93      0.68       208

   micro avg       0.55      0.55      0.55       398
   macro avg       0.59      0.53      0.45       398
weighted avg       0.59      0.55      0.46       398

0.550251256281407
 Performance report of  characters 1-gram with GaussianNB
              precision    recall  f1-score   support

           0       0.70      0.68      0.69       190
           1       0.72      0